This notebook implements crop field instance segmentation using predicted crop field extent and boundary probabilities.

## Load packages and modules

In [1]:
import numpy as np
import imageio.v2 as imageio
from osgeo import gdal
import os
from glob import glob
from skimage import measure
import pandas as pd
import subprocess
import sys

module_paths=['../1_Tune_hyperparameters_model_evaluation']
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

In [2]:
# import functions from modules
from datasets import export_geotiff
from instance_segment import InstSegm

## Define parameters

In [3]:
# input folder for predictions
input_folder='results'

# output folder to store instance segmentation results
out_folder=input_folder

# grid search results of thresholds
grd_search_df='../1_Tune_hyperparameters_model_evaluation/results/averaged/grid_search_thresholds.csv'

country = 'Mozambique'
str_year='2021'

## Do instance segmentation for all chunks and export as geotiff

In [4]:
# Load predicted exent and boundary files
files_extent_predictions=glob(input_folder+'/'+country+'_average_extent_prob_'+str_year+'*.tif')
print('Found {} field exent probability images'.format(len(files_extent_predictions)))

files_bound_predictions=glob(input_folder+'/'+country+'_average_bound_prob_'+str_year+'*.tif')
print('Found {} field boundary probability images'.format(len(files_bound_predictions)))

Found 1007 field exent probability images
Found 1007 field boundary probability images


In [5]:
# read in or provide the best thresholds
if not grd_search_df is None:
    hp_df=pd.read_csv(grd_search_df)
    t_ext_best=hp_df.iloc[hp_df['mIoU'].idxmax()]['t_ext']
    t_bnd_best=hp_df.iloc[hp_df['mIoU'].idxmax()]['t_bound']
else:
    t_ext_best=0.3
    t_bnd_best=0.1

In [6]:
%%time
instance_file_names=[] # exported field instance chunks
# loop through all averaged prediction chunks
for file_exent_prediction in files_extent_predictions:
    # extract geo information using gdal
    ds = gdal.Open(file_exent_prediction)
    geotrans=ds.GetGeoTransform()
    proj=ds.GetProjection()
    ds=None
    # corresponding boundary prediction
    file_bound_prediction=file_exent_prediction.replace('extent','bound')

    # read in files
    extent_prob=imageio.imread(file_exent_prediction)
    bound_prob=imageio.imread(file_bound_prediction)

    # do segmentation
    instances_predicted=InstSegm(extent_prob, bound_prob, t_ext=t_ext_best, t_bound=t_bnd_best)
    
    # label connected regions, non-field (-1) will be labelled as 0
    instances_predicted= measure.label(instances_predicted, background=-1,return_num=False)

    # export as geotiff
    outname=os.path.join(out_folder,os.path.basename(file_exent_prediction).replace('extent_prob','field_instance'))
    instance_file_names.append(outname)
    if not os.path.exists(outname):
        export_geotiff(outname,instances_predicted,geotrans,proj,gdal.GDT_Int16)

CPU times: user 1min 56s, sys: 1.93 s, total: 1min 58s
Wall time: 1min 23s
